In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from __future__ import print_function
from functools import reduce
import re
import tarfile
import numpy as np
from keras.utils.data_utils import get_file
from keras.utils import to_categorical
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import *
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ReduceLROnPlateau
print(keras.__version__)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'text.color':'white'})
import nltk
#%matplotlib inline
df_fcc = pd.read_csv("./comcast_fcc_complaints_2015.csv")

df_fcc.head()

Using TensorFlow backend.


2.0.5


,Ticket #,Customer Complaint,Date,Time,Received Via,City,State,Zip code,Status,Filing on Behalf of Someone,Description
0,250635,Comcast Cable Internet Speeds,4/22/2015,3:53:50 PM,Internet,Abingdon,Maryland,21009,Closed,No,I have been contacting Comcast Internet Techni...
1,223441,Payment disappear - service got disconnected,4/8/2015,10:22:56 AM,Internet,Acworth,Georgia,30102,Closed,No,Back in January 2015 I made 2 payments: One fo...
2,242732,Speed and Service,4/18/2015,9:55:47 AM,Internet,Acworth,Georgia,30101,Closed,Yes,Our home is located at in Acworth Georgia 3010...
3,277946,Comcast Imposed a New Usage Cap of 300GB that ...,5/7/2015,11:59:35 AM,Internet,Acworth,Georgia,30101,Open,Yes,Comcast in the Atlanta area has just put into ...
4,307175,Comcast not working and no service to boot,5/26/2015,1:25:26 PM,Internet,Acworth,Georgia,30101,Solved,No,I have been a customer of Comcast of some sort...


In [2]:
stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This','i','you','have','been','they','my','are','on','into','comcast','time','customer','even','now','company',
            'day','someone','thing','also','got','way','call','called','one','said','tell','service']
def text_to_wordlist(text, remove_stop_words=False, stem_words=False):
    # Clean the text, with the option to remove stop_words and to stem words.

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", " what is ", text)
    text = re.sub(r"What's", " what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r"switzerland", "Switzerland", text)
    text = re.sub(r"china", "China", text)
    text = re.sub(r"chinese", "Chinese", text) 
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text) 
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iPhone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the US", "America", text)
    text = re.sub(r"Astrology", "astrology", text)
    text = re.sub(r"Method", "method", text)
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    #text = re.sub(r"comcast", " singtel ", text)
    
    # Remove punctuation from text
    #text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [3]:
import re
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
xtrain =[]
ytrain =[]
ytrainDecIn =[]
ytrainDecOut =[]
for caseID in range(df_fcc.shape[0]):
    out = df_fcc['Description'][caseID].split('Follow-up message by FCC')
    inputString = re.sub(r"[^A-Za-z0-9.$ ]",'', out[0]).lower() 
    inputString = re.sub(' +',' ',inputString)
    inputString = text_to_wordlist(inputString,remove_stop_words=True)
    inputString = re.sub(r"[0-9]",'# ', inputString)+ ' \n'
    xtrain += [inputString]
    outputString = re.sub(r"[^A-Za-z0-9.$ ]",'', df_fcc['Customer Complaint'][caseID]).lower()
    outputString = text_to_wordlist(outputString,remove_stop_words=True)
    ytrain += [outputString]

print(xtrain[:10])
print(ytrain[:10])


['contacting internet technical support last # #  months resolve slow internet speed issues with no permanent resolution sent techs residence replaced hardware pay additional monthly fee services not received them when speak management very arrogant rude their tone demeaning unapologetic resolution problem never reached most get hung up experience long hold times complaint paying services am not receiving pay high speed internet boost internet speeds reliable acceptable technical support do not receive any above feel like duped paying services without actually receiving them can accurately say last # #  months paid # #  month high speed internet not received anything remotely close high speed internet every contacted made it extremely clear am not receiving services am paying continues bill me services am not receiving \n', 'back in january # # # #  made #  payments januarys februarys advance payment in march go web page make payment see account shows past due find out where payment we

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_SEQUENCE_LENGTH = 79
MAX_OUT_SEQUENCE_LENGTH = 8
MAX_NB_WORDS = 1500
EMBEDDING_DIM = 100 

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~')
tokenizer.fit_on_texts(['\t','\n']+xtrain +ytrain)
sequencestr = tokenizer.texts_to_sequences(xtrain)
sequencestt = tokenizer.texts_to_sequences(ytrain)
word_index = tokenizer.word_index
TOTAL_WORD_COUNT = len(word_index)
print('Found %s unique tokens.' % len(word_index))
print(word_index['\n'])
print(word_index['\t'])
word_index['\t']=1500

Found 10782 unique tokens.
10
6202


In [5]:
'''decodeInp = []
decodeOut = []
for i in sequencestt:
    decodeOut +=[i + [word_index['\n']]]
    decodeInp += [[word_index['\t']]+i]
print(decodeInp[0])
word_index['\n']'''

"decodeInp = []\ndecodeOut = []\nfor i in sequencestt:\n    decodeOut +=[i + [word_index['\n']]]\n    decodeInp += [[word_index['\t']]+i]\nprint(decodeInp[0])\nword_index['\n']"

In [6]:
datatr = pad_sequences(sequencestr, maxlen=MAX_SEQUENCE_LENGTH,padding='pre')
datatt = pad_sequences(sequencestt, maxlen=MAX_OUT_SEQUENCE_LENGTH-1,padding='post')
datattInp= []
datattOut = []
print(word_index['\n'])
for i in datatt:
    datattOut +=[list(i) + [word_index['\n']]]
    datattInp += [[word_index['\t']]+list(i)]
    print(datattInp[-1])
datattOut = np.array(datattOut) 
datattInp = np.array(datattInp) 
print(datatr.shape)
print(np.array(datattInp).shape)
print(np.array(datattOut).shape)

10
[1500, 27, 5, 82, 0, 0, 0, 0]
[1500, 201, 272, 0, 0, 0, 0, 0]
[1500, 37, 0, 0, 0, 0, 0, 0]
[1500, 1312, 47, 143, 86, 370, 0, 0]
[1500, 4, 189, 11, 0, 0, 0, 0]
[1500, 228, 1233, 42, 919, 7, 483, 175]
[1500, 423, 1332, 42, 214, 0, 0, 0]
[1500, 596, 94, 311, 0, 0, 0, 0]
[1500, 613, 0, 0, 0, 0, 0, 0]
[1500, 1264, 398, 4, 57, 178, 372, 122]
[1500, 54, 34, 1, 79, 8, 272, 0]
[1500, 1047, 1299, 113, 0, 0, 0, 0]
[1500, 1415, 28, 921, 340, 105, 197, 1417]
[1500, 5, 37, 0, 0, 0, 0, 0]
[1500, 5, 1375, 114, 645, 0, 0, 0]
[1500, 5, 88, 0, 0, 0, 0, 0]
[1500, 5, 1115, 37, 0, 0, 0, 0]
[1500, 3, 922, 110, 359, 95, 53, 13]
[1500, 658, 5, 0, 0, 0, 0, 0]
[1500, 1047, 192, 45, 20, 303, 0, 0]
[1500, 5, 33, 40, 270, 0, 0, 0]
[1500, 658, 27, 0, 0, 0, 0, 0]
[1500, 37, 0, 0, 0, 0, 0, 0]
[1500, 270, 324, 0, 0, 0, 0, 0]
[1500, 0, 0, 0, 0, 0, 0, 0]
[1500, 915, 355, 0, 0, 0, 0, 0]
[1500, 0, 0, 0, 0, 0, 0, 0]
[1500, 384, 116, 7, 0, 0, 0, 0]
[1500, 337, 612, 306, 5, 54, 81, 0]
[1500, 391, 5, 4, 189, 224, 0, 0]
[150

[1500, 5, 82, 0, 0, 0, 0, 0]
[1500, 1020, 0, 0, 0, 0, 0, 0]
[1500, 44, 54, 0, 0, 0, 0, 0]
[1500, 5, 27, 32, 0, 0, 0, 0]
[1500, 5, 148, 490, 0, 0, 0, 0]
[1500, 423, 5, 37, 0, 0, 0, 0]
[1500, 141, 483, 102, 11, 963, 0, 0]
[1500, 613, 0, 0, 0, 0, 0, 0]
[1500, 54, 0, 0, 0, 0, 0, 0]
[1500, 102, 4, 1253, 0, 0, 0, 0]
[1500, 522, 0, 0, 0, 0, 0, 0]
[1500, 5, 0, 0, 0, 0, 0, 0]
[1500, 522, 0, 0, 0, 0, 0, 0]
[1500, 148, 0, 0, 0, 0, 0, 0]
[1500, 141, 527, 375, 1091, 6, 1, 571]
[1500, 81, 0, 0, 0, 0, 0, 0]
[1500, 5, 490, 0, 0, 0, 0, 0]
[1500, 0, 0, 0, 0, 0, 0, 0]
[1500, 206, 143, 0, 0, 0, 0, 0]
[1500, 42, 214, 0, 0, 0, 0, 0]
[1500, 54, 324, 0, 0, 0, 0, 0]
[1500, 117, 319, 0, 0, 0, 0, 0]
[1500, 5, 37, 0, 0, 0, 0, 0]
[1500, 80, 929, 627, 15, 0, 0, 0]
[1500, 1357, 0, 0, 0, 0, 0, 0]
[1500, 191, 171, 1, 427, 266, 2, 0]
[1500, 141, 423, 67, 0, 0, 0, 0]
[1500, 562, 78, 0, 0, 0, 0, 0]
[1500, 81, 0, 0, 0, 0, 0, 0]
[1500, 11, 1002, 150, 69, 263, 0, 0]
[1500, 353, 54, 0, 0, 0, 0, 0]
[1500, 1276, 0, 0, 0, 0, 0,

In [7]:
[list(datatt[0])+[11]]

[[27, 5, 82, 0, 0, 0, 0, 11]]

In [8]:
a = [1,2]
a = a + [word_index['\n']]
a

[1, 2, 10]

In [9]:
word_index.items()

dict_items([('was', 1), ('in', 2), ('me', 3), ('not', 4), ('internet', 5), ('it', 6), ('with', 7), ('be', 8), ('would', 9), ('\n', 10), ('no', 11), ('we', 12), ('had', 13), ('at', 14), ('from', 15), ('told', 16), ('up', 17), ('bill', 18), ('when', 19), ('am', 20), ('their', 21), ('month', 22), ('has', 23), ('he', 24), ('get', 25), ('only', 26), ('cable', 27), ('by', 28), ('do', 29), ('them', 30), ('our', 31), ('phone', 32), ('out', 33), ('after', 34), ('she', 35), ('account', 36), ('speed', 37), ('there', 38), ('modem', 39), ('all', 40), ('will', 41), ('data', 42), ('pay', 43), ('over', 44), ('services', 45), ('back', 46), ('new', 47), ('again', 48), ('did', 49), ('can', 50), ('were', 51), ('issue', 52), ('never', 53), ('billing', 54), ('could', 55), ('more', 56), ('being', 57), ('still', 58), ('since', 59), ('months', 60), ('home', 61), ('mbps', 62), ('other', 63), ('times', 64), ('any', 65), ('paying', 66), ('tv', 67), ('received', 68), ('price', 69), ('charge', 70), ('problem', 71),

In [10]:
import os

embeddings_index = {}
f = open(os.path.join(r'C:\Users\HuaSheng\Desktop\reddragonai\dl_dev_course-master\redaicse\LSTMProject\glove6B100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [11]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
print(embedding_matrix.shape)

(10783, 100)


In [12]:
from keras.layers import Embedding

embedding_layerIn = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

embedding_layerOut = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_OUT_SEQUENCE_LENGTH,
                            trainable=False)

In [13]:
print(datattOut.shape)

(2225, 8)


In [14]:
from keras.utils import to_categorical
def token2OHE(xqo):
    temp = np.zeros((xqo.shape[0], MAX_OUT_SEQUENCE_LENGTH,MAX_NB_WORDS),dtype='float32')
    for i,idx in enumerate(xqo):
        #print(idx.shape)
        for u,v in enumerate(idx):
            temp [i,u, v] = 1.
    return temp
datattOutOHE=token2OHE(datattOut)

In [15]:
print(datattOutOHE.shape)
print(datatr.shape)
print(datattInp.shape)

(2225, 8, 1500)
(2225, 79)
(2225, 8)


In [16]:
LSTM_SZ = 49
encoder_inputs = Input(shape=( MAX_SEQUENCE_LENGTH ,),name='encoder_inputs') 
print(encoder_inputs)
eInp =embedding_layerIn(encoder_inputs)
print(eInp)
encoder = LSTM(LSTM_SZ ,return_state=True,unroll=False)
encoder_outputs, state_h, state_c = encoder(eInp)
encoder_states = [state_h, state_c]
print(encoder_states)
decoder_inputs = Input(shape=(MAX_OUT_SEQUENCE_LENGTH,),name='decoder_inputs')
print(decoder_inputs)
dInp = embedding_layerOut(decoder_inputs)
print(dInp)
decoder_lstm = LSTM(LSTM_SZ , return_sequences=True, return_state=True,name='LSTM_01') 
decoder_output, _, _ = decoder_lstm(dInp , initial_state=encoder_states)
print('wow'+str(decoder_output))

decoder_dense =Dense(MAX_NB_WORDS, activation='softmax')
decoder_outputs = decoder_dense(decoder_output)
print(decoder_outputs)

Tensor("encoder_inputs:0", shape=(?, 79), dtype=float32)
Tensor("embedding_1/Gather:0", shape=(?, 79, 100), dtype=float32)
[<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 49) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 49) dtype=float32>]
Tensor("decoder_inputs:0", shape=(?, 8), dtype=float32)
Tensor("embedding_2/Gather:0", shape=(?, 8, 100), dtype=float32)
wowTensor("LSTM_01/transpose_1:0", shape=(?, ?, 49), dtype=float32)
Tensor("dense_1/truediv:0", shape=(?, 8, 1500), dtype=float32)


In [17]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
reduce_LR = ReduceLROnPlateau(monitor='val_loss',factor = 0.9, patience=3,cooldown=2, min_lr = 0.00001)
model.fit([datatr, datattInp], datattOutOHE,
          batch_size=256,
          epochs=240,
          callbacks=[reduce_LR],
          validation_split=0.05)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
encoder_inputs (InputLayer)      (None, 79)            0                                            
____________________________________________________________________________________________________
decoder_inputs (InputLayer)      (None, 8)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 79, 100)       1078300     encoder_inputs[0][0]             
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 8, 100)        1078300     decoder_inputs[0][0]             
___________________________________________________________________________________________

2113/2113 [==============================] - 4s - loss: 1.8517 - acc: 0.6857 - val_loss: 2.1002 - val_acc: 0.6853
Epoch 49/240
2113/2113 [==============================] - 4s - loss: 1.8501 - acc: 0.6835 - val_loss: 2.0869 - val_acc: 0.6842
Epoch 50/240
2113/2113 [==============================] - 4s - loss: 1.8387 - acc: 0.6859 - val_loss: 2.0897 - val_acc: 0.6853
Epoch 51/240
2113/2113 [==============================] - 4s - loss: 1.8365 - acc: 0.6842 - val_loss: 2.0761 - val_acc: 0.6864
Epoch 52/240
2113/2113 [==============================] - 4s - loss: 1.8204 - acc: 0.6882 - val_loss: 2.1160 - val_acc: 0.6585
Epoch 53/240
2113/2113 [==============================] - 4s - loss: 1.8205 - acc: 0.6832 - val_loss: 2.0707 - val_acc: 0.6864
Epoch 54/240
2113/2113 [==============================] - 4s - loss: 1.8204 - acc: 0.6847 - val_loss: 2.0680 - val_acc: 0.6853
Epoch 55/240
2113/2113 [==============================] - 4s - loss: 1.7971 - acc: 0.6898 - val_loss: 2.0905 - val_acc: 0.67

2113/2113 [==============================] - 4s - loss: 1.5846 - acc: 0.6892 - val_loss: 1.9530 - val_acc: 0.6942
Epoch 113/240
2113/2113 [==============================] - 4s - loss: 1.5460 - acc: 0.7060 - val_loss: 1.9496 - val_acc: 0.6998
Epoch 114/240
2113/2113 [==============================] - 4s - loss: 1.5424 - acc: 0.7074 - val_loss: 1.9502 - val_acc: 0.6953
Epoch 115/240
2113/2113 [==============================] - 4s - loss: 1.5409 - acc: 0.7060 - val_loss: 1.9558 - val_acc: 0.6920
Epoch 116/240
2113/2113 [==============================] - 4s - loss: 1.5448 - acc: 0.7052 - val_loss: 2.0481 - val_acc: 0.6261
Epoch 117/240
2113/2113 [==============================] - 4s - loss: 1.5437 - acc: 0.6991 - val_loss: 1.9545 - val_acc: 0.6908
Epoch 118/240
2113/2113 [==============================] - 3s - loss: 1.5301 - acc: 0.7072 - val_loss: 1.9552 - val_acc: 0.6920
Epoch 119/240
2113/2113 [==============================] - 4s - loss: 1.5252 - acc: 0.7074 - val_loss: 1.9444 - val_ac

2113/2113 [==============================] - 4s - loss: 1.3999 - acc: 0.7172 - val_loss: 1.9355 - val_acc: 0.6975
Epoch 177/240
2113/2113 [==============================] - 4s - loss: 1.3983 - acc: 0.7168 - val_loss: 1.9350 - val_acc: 0.6987
Epoch 178/240
2113/2113 [==============================] - 4s - loss: 1.3968 - acc: 0.7163 - val_loss: 1.9357 - val_acc: 0.6964
Epoch 179/240
2113/2113 [==============================] - 4s - loss: 1.3948 - acc: 0.7167 - val_loss: 1.9409 - val_acc: 0.6975
Epoch 180/240
2113/2113 [==============================] - 4s - loss: 1.3940 - acc: 0.7172 - val_loss: 1.9359 - val_acc: 0.6964
Epoch 181/240
2113/2113 [==============================] - 4s - loss: 1.3932 - acc: 0.7175 - val_loss: 1.9385 - val_acc: 0.6975
Epoch 182/240
2113/2113 [==============================] - 4s - loss: 1.3916 - acc: 0.7176 - val_loss: 1.9385 - val_acc: 0.6942
Epoch 183/240
2113/2113 [==============================] - 3s - loss: 1.3909 - acc: 0.7188 - val_loss: 1.9397 - val_ac

2113/2113 [==============================] - 4s - loss: 1.3535 - acc: 0.7210 - val_loss: 1.9427 - val_acc: 0.6987


In [18]:
model.fit([datatr, datattInp], datattOutOHE,
          batch_size=32,
          epochs=1,
          callbacks=[reduce_LR],
          validation_split=0.1)

Train on 2002 samples, validate on 223 samples
Epoch 1/1
2002/2002 [==============================] - 9s - loss: 1.6060 - acc: 0.7041 - val_loss: 1.7639 - val_acc: 0.7029


In [18]:
inv_map = {v: k for k, v in word_index.items()}
inv_map[0]=' '

In [19]:
stringInp =[]
for i in datatr[0]:
    stringInp +=[inv_map[i]]
print(stringInp)

['their', 'resolution', 'problem', 'never', 'reached', 'most', 'get', 'hung', 'up', 'experience', 'long', 'hold', 'times', 'complaint', 'paying', 'services', 'am', 'not', 'receiving', 'pay', 'high', 'speed', 'internet', 'boost', 'internet', 'speeds', 'reliable', 'acceptable', 'technical', 'support', 'do', 'not', 'receive', 'any', 'above', 'feel', 'like', 'paying', 'services', 'without', 'actually', 'receiving', 'them', 'can', 'say', 'last', 'months', 'paid', 'month', 'high', 'speed', 'internet', 'not', 'received', 'anything', 'close', 'high', 'speed', 'internet', 'every', 'contacted', 'made', 'it', 'extremely', 'clear', 'am', 'not', 'receiving', 'services', 'am', 'paying', 'continues', 'bill', 'me', 'services', 'am', 'not', 'receiving', '\n']


In [20]:
encoder_model = Model(encoder_inputs, encoder_states)
dataSpace=encoder_model.predict(datatr)
print(dataSpace[0].shape)

(2225, 49)


In [21]:
outVect = np.concatenate((dataSpace[0],dataSpace[1]),axis=1)
print(outVect.shape)

(2225, 98)


In [22]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(outVect)


In [23]:
'''print(X_embedded.shape)
print(datattOut[:,1]/1500)
plt.figure(figsize=(20,10))
plt.scatter(X_embedded[:,0],X_embedded[:,1],c=np.divide(datattOut[:,1],1500))
plt.show'''

'print(X_embedded.shape)\nprint(datattOut[:,1]/1500)\nplt.figure(figsize=(20,10))\nplt.scatter(X_embedded[:,0],X_embedded[:,1],c=np.divide(datattOut[:,1],1500))\nplt.show'

In [24]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)

labelsText =[]
for i in datattOut[:,0:8]:
    someText =[]
    for u in i:
        someText +=[inv_map[u]]
    #print(someText)
    labelsText+=[''.join(someText) ]

trace = Scatter(x = X_embedded[:,0],y = X_embedded[:,1],mode = 'markers+text',marker=dict(
        size='16',
        color = np.divide(datattOut[:,1],30), #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    ),text = labelsText,textfont=dict( size=12, color='#7f7f7f')
)
plot([trace])


'file://C:\\Users\\HuaSheng\\Desktop\\hdbHackerthon\\temp-plot.html'

In [ ]:
len(labelsText)

In [ ]:
dataSpace[0].shape

In [ ]:
np.array(dataSpace).shape
d = {'time': df_fcc['Time'],'date': df_fcc['Date'],'comments':xtrain,'headings':ytrain,'contentEncode':list(datattInp), 
     'headingEncode':list(datattOut),'encodedSpace1':list(dataSpace[0]),'encodedSpace2':list(dataSpace[1])}
df = pd.DataFrame(data=d)
df.to_csv('comcastSample.csv')

In [ ]:
def decode_sequence(input_seq,yIn,inv_map):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros(( 1, query_maxlen))
    target_seq[ 0, 0] = yIn
    stop_condition = False #our stop condition will be '/n' which acts as <EOS>
    decoded_sentence = ''
    ct =0
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        print([inv_map[i] for i in target_seq[0] ])
        target_seq[0, ct+1] = np.argmax(output_tokens[0,ct, :])
        sampled_char = inv_map[target_seq[0, ct+1]]
        decoded_sentence += ' '+sampled_char
        if (ct>6):
            stop_condition = True
        ct+=1
        states_value = [h, c]
    return decoded_sentence,target_seq
testIdx =0
input_seq = np.expand_dims(datatr[testIdx],axis=0)
yIn =
decoded_sentence,target_seq = decode_sequence(input_seq,yIn,inv_map)

In [ ]:
tokenizer.word_index['speed']

In [ ]:
sqrTtOut[:5]

In [ ]:
MAX_SEQUENCE_LENGTH = 300
datatr = pad_sequences(sqrTr , maxlen=MAX_SEQUENCE_LENGTH,padding='pre')
datatt = pad_sequences(sqrTt, maxlen=15,padding='post')
story_maxlen = max(map(len, datatr))
print('Shape of data tensor:', datatr1.shape)
print(datatt[0])
print(sqrTr[0])
len(tokenizer.word_index)
print(story_maxlen)

In [ ]:
tokenizer.word_index['\n']

In [ ]:
from wordcloud import WordCloud, STOPWORDS

list_stops = ('comcast','time','customer','even','now','company',
            'day','someone','thing','also','got','way','call','called','one','said','tell','service')

for word in list_stops:
    STOPWORDS.add(word)

In [ ]:
df_fcc['Customer Complaint'].value_counts()

In [ ]:
from wordcloud import STOPWORDS
common_complaints = df_fcc['Customer Complaint'].dropna().tolist()
common_complaints =''.join(common_complaints).lower()

list_stops = ('comcast','now','company','day','someone','thing','also','got','way','call','called','one','said','tell')

for word in list_stops:
    STOPWORDS.add(word)
    

In [ ]:
wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color='white',
                      width=1200,
                      height=1000).generate(common_complaints)
plt.imshow(wordcloud)
plt.title('Frequent words for customer complaints')
plt.axis('off')
plt.show()